In [1]:
from torch.autograd import Variable
from torch.utils import data
from typing import Any, Dict, List
import collections
import json
import numpy as np
import os
import pandas as pd
import pickle
import string
import tensorboardX
import torch

In [2]:
def load_data(file_path) -> pd.DataFrame:
    with open(file_path, 'r') as infile:
        data = infile.read().split('\n')

    data = list(map(json.loads, data[:-1]))

    df = pd.DataFrame(data)
    
    print(len(df))
    df = df.loc[df.gold_label != '-']
    print(len(df))
    return df

In [3]:
file_path_train = 'data/multinli_1.0_train.jsonl'
file_path_val = 'data/multinli_1.0_dev_matched.jsonl'

In [4]:
%%time
df_train = load_data(file_path_train)

392702
392702
CPU times: user 6.58 s, sys: 1.19 s, total: 7.77 s
Wall time: 7.77 s


In [5]:
%%time
df_val = load_data(file_path_val)

10000
9815
CPU times: user 328 ms, sys: 7.47 ms, total: 336 ms
Wall time: 332 ms


In [6]:
df_train.head()

,annotator_labels,genre,gold_label,pairID,promptID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse
0,[neutral],government,neutral,31193n,31193,Conceptually cream skimming has two basic dime...,( ( Conceptually ( cream skimming ) ) ( ( has ...,(ROOT (S (NP (JJ Conceptually) (NN cream) (NN ...,Product and geography are what make cream skim...,( ( ( Product and ) geography ) ( ( are ( what...,(ROOT (S (NP (NN Product) (CC and) (NN geograp...
1,[entailment],telephone,entailment,101457e,101457,you know during the season and i guess at at y...,( you ( ( know ( during ( ( ( the season ) and...,(ROOT (S (NP (PRP you)) (VP (VBP know) (PP (IN...,You lose the things to the following level if ...,( You ( ( ( ( lose ( the things ) ) ( to ( the...,(ROOT (S (NP (PRP You)) (VP (VBP lose) (NP (DT...
2,[entailment],fiction,entailment,134793e,134793,One of our number will carry out your instruct...,( ( One ( of ( our number ) ) ) ( ( will ( ( (...,(ROOT (S (NP (NP (CD One)) (PP (IN of) (NP (PR...,A member of my team will execute your orders w...,( ( ( A member ) ( of ( my team ) ) ) ( ( will...,(ROOT (S (NP (NP (DT A) (NN member)) (PP (IN o...
3,[entailment],fiction,entailment,37397e,37397,How do you know? All this is their information...,( ( How ( ( ( do you ) know ) ? ) ) ( ( All th...,(ROOT (S (SBARQ (WHADVP (WRB How)) (SQ (VBP do...,This information belongs to them.,( ( This information ) ( ( belongs ( to them )...,(ROOT (S (NP (DT This) (NN information)) (VP (...
4,[neutral],telephone,neutral,50563n,50563,yeah i tell you what though if you go price so...,( yeah ( i ( ( tell you ) ( what ( ( though ( ...,(ROOT (S (VP (VB yeah) (S (NP (FW i)) (VP (VB ...,The tennis shoes have a range of prices.,( ( The ( tennis shoes ) ) ( ( have ( ( a rang...,(ROOT (S (NP (DT The) (NN tennis) (NNS shoes))...


In [7]:
df_val.head()

,annotator_labels,genre,gold_label,pairID,promptID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse
0,"[neutral, entailment, neutral, neutral, neutral]",slate,neutral,63735n,63735,The new rights are nice enough,( ( The ( new rights ) ) ( are ( nice enough )...,(ROOT (S (NP (DT The) (JJ new) (NNS rights)) (...,Everyone really likes the newest benefits,( Everyone ( really ( likes ( the ( newest ben...,(ROOT (S (NP (NN Everyone)) (VP (ADVP (RB real...
1,"[contradiction, contradiction, contradiction, ...",government,contradiction,91383c,91383,This site includes a list of all award winners...,( ( This site ) ( ( includes ( ( ( ( a list ) ...,(ROOT (S (NP (DT This) (NN site)) (VP (VBZ inc...,The Government Executive articles housed on th...,( ( ( The ( Government ( Executive articles ) ...,(ROOT (S (NP (NP (DT The) (NNP Government) (NN...
2,"[entailment, entailment, entailment, entailmen...",telephone,entailment,755e,755,uh i don't know i i have mixed emotions about ...,( ( ( ( uh ( i ( ( do n't ) ( know ( ( i i ) (...,(ROOT (SINV (S (S (INTJ (UH uh)) (NP (FW i)) (...,"I like him for the most part, but would still ...",( I ( ( ( ( ( ( like him ) ( for ( the ( most ...,(ROOT (S (NP (PRP I)) (VP (VP (VBP like) (NP (...
3,"[contradiction, contradiction, contradiction, ...",telephone,contradiction,78013c,78013,yeah i i think my favorite restaurant is alway...,( yeah ( ( i i ) ( think ( ( my ( favorite res...,(ROOT (S (VP (VB yeah) (NP (NP (FW i) (FW i)) ...,My favorite restaurants are always at least a ...,( ( My ( favorite restaurants ) ) ( ( ( ( are ...,(ROOT (S (NP (PRP$ My) (JJ favorite) (NNS rest...
4,"[contradiction, contradiction, contradiction, ...",telephone,contradiction,96377c,96377,i don't know um do you do a lot of camping,( i ( ( do n't ) ( know ( um ( do ( you ( do (...,(ROOT (S (NP (FW i)) (VP (VBP do) (RB n't) (VP...,I know exactly.,( I ( ( know exactly ) . ) ),(ROOT (S (NP (PRP I)) (VP (VBP know) (ADVP (RB...


In [8]:
df_train.head(10)[['sentence1', 'sentence2', 'gold_label']]

,sentence1,sentence2,gold_label
0,Conceptually cream skimming has two basic dime...,Product and geography are what make cream skim...,neutral
1,you know during the season and i guess at at y...,You lose the things to the following level if ...,entailment
2,One of our number will carry out your instruct...,A member of my team will execute your orders w...,entailment
3,How do you know? All this is their information...,This information belongs to them.,entailment
4,yeah i tell you what though if you go price so...,The tennis shoes have a range of prices.,neutral
5,my walkman broke so i'm upset now i just have ...,I'm upset that my walkman broke and now I have...,entailment
6,But a few Christian mosaics survive above the ...,Most of the Christian mosaics were destroyed b...,neutral
7,(Read for Slate 's take on Jackson's findings.),Slate had an opinion on Jackson's findings.,entailment
8,Gays and lesbians.,Heterosexuals.,contradiction
9,At the end of Rue des Francs-Bourgeois is what...,Place des Vosges is constructed entirely of gr...,contradiction


# CONFIG

In [9]:
BATCH_SIZE = 1

# Data loader

In [10]:
id_to_lbl = ['neutral', 'entailment', 'contradiction']
lbl_to_id = {
    lbl: ix
    for ix, lbl in enumerate(id_to_lbl)
}

In [11]:
class MNLIDataset(data.Dataset):
    def __init__(self, df: pd.DataFrame, word_vectors: Dict) -> None:
        df['label_id'] = df.gold_label.map(lbl_to_id)
        df['sentence1'] = df['sentence1'].apply(lambda x: x.strip(string.punctuation))
        df['sentence2'] = df['sentence2'].apply(lambda x: x.strip(string.punctuation))

        self.df = df
        self.word_vectors = word_vectors
        
    def __len__(self) -> int:
        return len(self.df)

    def _sentence_to_vec(self, sentence: str) -> np.ndarray:
        # FIXME
        vectors = []
        final_sentence = []
        for word in sentence.split(' '):
            if word not in self.word_vectors:
                continue

            final_sentence.append(word)
            vectors.append(self.word_vectors[word])

#         vectors = np.vstack(vectors)
        vectors = np.array(vectors)
        final_sentence = ' '.join(final_sentence)
        return vectors, final_sentence
        
    def _preprocess(self, record):
        # Convert sentences to word vectors, return list of 
        v1, fs1 = self._sentence_to_vec(record['sentence1'])
        v2, fs2 = self._sentence_to_vec(record['sentence2'])
        return {
            'sentence1': v1,
            'sentence2': v2,
            'label': record['label_id'],
            'final_sentence1': fs1,
            'final_sentence2': fs2,
        }

    def __getitem__(self, ix):
        return self._preprocess(self.df.iloc[ix])

In [12]:
def load_word_vectors(file_path: str) -> Dict:
    with open(file_path, 'r') as infile:
        data_glove = infile.read().split('\n')

    data_glove = map(lambda x: x.split(), data_glove)  # Split the words

    glove = {
        line[0]: np.array(line[1:], dtype=np.float32)
        for line in data_glove
        if len(line) == 301
    }

    return glove

In [18]:
%%time
pickle_file = 'models/glove.pickle'
if not os.path.exists(pickle_file):
    glove = load_word_vectors('models/glove.840B.300d.txt')  # FIXME: There shold be 2196017 words
    print(len(glove))

    with open(pickle_file, 'wb') as outfile:
        pickle.dump(glove, outfile)

CPU times: user 61 µs, sys: 25 µs, total: 86 µs
Wall time: 67 µs


In [19]:
%%time
with open(pickle_file, 'rb') as infile:
    glove = pickle.load(infile)

CPU times: user 6.39 s, sys: 3.44 s, total: 9.83 s
Wall time: 9.84 s


### Get the dataloader!

In [32]:
dataset_train = MNLIDataset(df_train, word_vectors=glove)
dataloader_train = data.DataLoader(dataset=dataset_train, batch_size=BATCH_SIZE, pin_memory=True, shuffle=True)

In [33]:
dataset_val = MNLIDataset(df_val, word_vectors=glove)
dataloader_val = data.DataLoader(dataset=dataset_val, batch_size=BATCH_SIZE, pin_memory=True, shuffle=True)

In [34]:
print(len(dataset_train), len(dataloader_train))
print(len(dataset_val), len(dataloader_val))

392702 392702
9815 9815


### Class Weights

In [23]:
# We have balanced classes
df_train.gold_label.value_counts()

contradiction    130903
neutral          130900
entailment       130899
Name: gold_label, dtype: int64

In [24]:
# We have balanced classes
df_val.gold_label.value_counts()

entailment       3479
contradiction    3213
neutral          3123
Name: gold_label, dtype: int64

## Basic network

In [25]:
import torch.nn.functional as F

In [27]:
class CBOW(torch.nn.Module):
    def __init__(self, vector_dim=300, n_out=3) -> None:
        super().__init__()
        self.classifier = torch.nn.Linear(vector_dim * 3, n_out)
        
        self.vector_dim = vector_dim

    def forward(self, data):
        vectors1, vectors2 = data
#         print(vectors1.shape, vectors2.shape)
        s1, s2 = torch.sum(vectors1, dim=1), torch.sum(vectors2, dim=1)
#         print(s1.shape, s2.shape)
        
        features = torch.cat([s1-s2, s1+s2, s1*s2], dim=1)
#         print(features.shape)
        pred = self.classifier(features)
        return F.log_softmax(pred, dim=1)

## Train

In [28]:
def prep_inputs(batch, device=0):
    inputs = (Variable(batch['sentence1']), Variable(batch['sentence2']))
    label = Variable(batch['label'])

    return inputs, label

def evaluate(model, dataloader, loss_func, device, n_batches=10):
    loss = 0
    accuracy = 0

    for ix, batch in enumerate(dataloader):
        if ix >= n_batches:
            break
        
        inputs, label = prep_inputs(batch)
        
        predicted = model(inputs)

        loss += loss_func(predicted, label)
        prediction = torch.argmax(predicted, dim=1)

        # TODO: get the confusion here
        accuracy += (torch.sum(prediction == label).data.cpu().numpy() / len(prediction))

    accuracy /= n_batches
    loss /= n_batches
    
    return loss, accuracy

In [35]:
n_epochs = 8
val_every = 1000

DEVICE=1

In [36]:
model = CBOW()

loss_func = torch.nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-8, momentum=0)

In [37]:
epoch = 0

In [38]:
model_str = 'cbow-0.1'
model_dir = 'models/{}'.format(model_str)
log_dir = 'logs/{}'.format(model_str)

os.makedirs(model_dir)
writer = tensorboardX.SummaryWriter(log_dir)

In [39]:
model.train(True)
while epoch < n_epochs:
    
    for iteration, batch in enumerate(dataloader_train):
        inputs, label = prep_inputs(batch)

        predicted = model(inputs)

        optimizer.zero_grad()
        loss_train = loss_func(predicted, label)
        loss_train.backward()
        optimizer.step()

        # Log to tensorboard
        iter_total = (epoch * len(dataloader_train)) + iteration
        writer.add_scalar('train.loss', loss_train.data.cpu().numpy(), iter_total)

        # Calculate validation accuracy
        if iteration % val_every == 0:
            loss_val, acc_val = evaluate(model, dataloader_val, device=DEVICE, loss_func=loss_func, n_batches=10)

            s = "Epoch: {}, {:.2f}%: train loss: {}, validation loss: {}, validation acc: {}".format(
                epoch, (iteration / len(dataloader_train)) * 100, loss_train.data.cpu().numpy(), loss_val, acc_val
            )
            print(s)

    print('\n------------------------------------------------------------------------------------------------------------')
    print("Epoch:", epoch + 1, "label accuracy:", accuracy_val)
    print('------------------------------------------------------------------------------------------------------------\n')

    torch.save(model.state_dict(), f=os.path.join(model_dir, '{}_{}_{}.pt'.format(model_str, epoch, iteration)))
    epoch += 1

Epoch: 0, 0.00%: train loss: 16.436447143554688, validation loss: 35.54566955566406
label accuracy: 0.4
Epoch: 0, 0.25%: train loss: 0.33871355652809143, validation loss: 2.266204357147217
label accuracy: 0.5
Epoch: 0, 0.51%: train loss: 11.131587028503418, validation loss: 14.522628784179688
label accuracy: 0.3


RuntimeError: cannot unsqueeze empty tensor

## Test